# Live demo: Processing gravity data with Fatiando a Terra

## Import packages

In [ ]:
import pygmt
import pyproj
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

import pooch
import verde as vd
import boule as bl
import harmonica as hm

## Load Bushveld Igneous Complex gravity data (South Africa) and a DEM

In [ ]:
url = "https://github.com/fatiando/2021-gsh/main/raw/notebook/data/bushveld_gravity.csv"

In [ ]:
data = pd.read_csv(fname)
data

In [ ]:
# Set projection for all plots:
projection_gmt="M15c"

# Obtain the region to plot using Verde ([W, E, S, N])
region_deg = vd.get_region((data.longitude, data.latitude))

# Plot Earth relief
fig = pygmt.Figure()
fig.grdimage(
    "@earth_relief_01m",
    region=region_deg,
    projection=projection_gmt,
    shading="+a45+nt0.7",
    cmap="gray",
)

# Plot the observerd gravity using pygmt
fig.coast(shorelines=True, region=region_deg)
pygmt.makecpt(cmap="viridis", series=[data.gravity.min(), data.gravity.max()])
fig.plot(
    x=data.longitude, y=data.latitude, color=data.gravity, cmap=True, style="c4p", pen='black', projection=projection_gmt
)
fig.basemap(projection=projection_gmt, frame=True)
fig.colorbar(frame='af+l"Observed Gravity [mGal]"')
fig.show()

In [ ]:
url = "https://github.com/fatiando/transform21/raw/main/data/bushveld_topography.nc"
fname = pooch.retrieve(url, known_hash=None, fname="bushveld_topography.nc")
fname

In [ ]:
topography = xr.load_dataarray(fname)
topography

In [ ]:
# Plot topography using pygmt
fig = pygmt.Figure()

topo_region = vd.get_region((topography.longitude.values, topography.latitude.values))
fig.coast(shorelines=True, region=topo_region)

vmin, vmax = topography.values.min(), topography.values.max()
pygmt.makecpt(cmap="batlow", series=[vmin, vmax])
fig.grdimage(topography, projection=projection_gmt)
fig.basemap(projection=projection_gmt, frame=True)
fig.colorbar(frame='af+l"Topography [m]"')
fig.show()

## Compute gravity disturbance

In [ ]:
data["disturbance"] = data.gravity - normal_gravity
data

In [ ]:
fig = pygmt.Figure()
maxabs = vd.maxabs(data.disturbance)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.disturbance,
    cmap=True,
    style="c4p",
    projection=projection_gmt,
    region=region_deg,
)
fig.basemap(projection=projection_gmt, frame=True)
fig.colorbar(frame='af+l"Gravity disturbance [mGal]"')
fig.show()

## Remove terrain effect

### Project the data to plain coordinates

In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=data.latitude.mean())

In [ ]:
data["easting"] = easting
data["northing"] = northing
data

### Project the topography to plain coordinates

### Compute gravitational effect of the layer of prisms

Create a model of the terrain with prisms

![](images/prisms.svg)

In [ ]:
data["bouguer"] = data.disturbance - terrain_effect
data

In [ ]:
fig = pygmt.Figure()
fig.coast(shorelines=True, region=region_deg)
maxabs = vd.maxabs(data.bouguer)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.bouguer,
    cmap=True,
    style="c4p",
    projection=projection_gmt,
    region=region_deg,
)
fig.basemap(projection=projection_gmt, frame=True)
fig.colorbar(frame='af+l"Bouguer gravity disturbance [mGal]"')
fig.show()

## Calculate residuals

In [ ]:
data["residuals"] = residuals
data

In [ ]:
fig = pygmt.Figure()
maxabs = np.quantile(np.abs(data.residuals), 0.99)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.residuals,
    cmap=True,
    style="c5p",
    projection=projection_gmt,
    region=region_deg,
)
fig.basemap(projection=projection_gmt, frame=True)
fig.colorbar(frame='af+l"Bouguer gravity disturbance [mGal]"')
fig.show()

## Grid the residuals with Equivalent Sources

We can use [Harmonica](https://www.fatiando.org/harmonica) to grid the residuals though the equivalent sources technique

![](images/eql.svg)

In [ ]:
fig = pygmt.Figure()
scale = np.quantile(np.abs(grid.residuals), 0.995)
pygmt.makecpt(cmap="polar", series=[-scale, scale], no_bg=True)
fig.grdimage(
    grid.residuals,
    projection=projection_gmt,
    region=region_deg,
    shading="+a45+nt0.15",
    cmap=True,
)
fig.basemap(projection=projection_gmt, region=region_deg, frame=True)
fig.colorbar(frame='af+l"Residuals [m]"')
fig.show()